In [2]:
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

## Data Processing

In [3]:
##Minkowski product of 4-vectors p1, p2.
def m_prod_arr(p1, p2):
    return np.multiply(p1[:,0], p2[:,0]) - np.sum(np.multiply(p1[:,1:], p2[:,1:]), axis=1)

In [4]:
def data_preprocessing(me_filename, mom_filename, frac=1):
    ##Data Aquisition
    me_raw = np.load(me_filename, allow_pickle=True) #Matrix elements
    mom_raw = np.load(mom_filename, allow_pickle=True, encoding='bytes') #4-momenta of inputs
    mom_raw = np.array([np.array(element) for element in mom_raw])
    
    me_raw=me_raw[:int(frac*len(me_raw))]
    mom_raw=mom_raw[:int(frac*len(mom_raw))]
    
    p_12 = mom_raw[:,2] + mom_raw[:,3] #p1+p2
    p_13 = mom_raw[:,2] + mom_raw[:,4] #p1+p3
    p_23 = mom_raw[:,3] + mom_raw[:,4] #p2+p3

    s_12 = m_prod_arr(p_12, p_12) #center of mass energies of two of three beams
    s_13 = m_prod_arr(p_13, p_13)
    s_23 = m_prod_arr(p_23, p_23)
    
    ##Flatten Momentum
    mom = np.array([np.ndarray.flatten(np.array(element)[2:]) for element in mom_raw])

    ##Reformat Matrix Element (remove divergent behaviour)
    me = np.multiply(np.multiply(me_raw, s_13), s_23)/(s_12+s_13+s_23)
    
    return(me, mom)

In [5]:
##Obtain & process data
me_train, mom_train = data_preprocessing('LO_3_MG/3Jet_3000000.npy', 'LO_3_0.01_NJet/PS3_0.01_3000000.npy', frac=0.1)
me_test, mom_test = data_preprocessing('LO_3_MG/3Jet_500000.npy', 'LO_3_0.01_NJet/PS3_0.01_500000.npy', frac=0.1)

##Rescale
scaler = StandardScaler()

mom_train = scaler.fit_transform(mom_train) #Rescale on training set
mom_test = scaler.transform(mom_test) #Rescale on test set

## Machine Learning

Random Forest Model:

In [9]:
##Random Forest Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

param_grid = {'n_estimators': [10], 'max_features': [2] }

forest_reg = RandomForestRegressor() 

#Grid search through parameter space
rand_search = RandomizedSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
rand_search.fit(mom_train, me_train)

me_predict = rand_search.predict(mom_test)
forest_mse = mean_squared_error(me_test, me_predict)
print(np.sqrt(forest_mse))

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


0.0016273754965707213


Polynomial Fit:

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet

poly_features = PolynomialFeatures(degree=2)
poly_train = poly_features.fit_transform(mom_train)
poly_test = poly_features.transform(mom_test)

linreg = LinearRegression().fit(poly_train, me_train)

me_predict_lin = linreg.predict(poly_test) #Prediction on test set
lin_mse = mean_squared_error(me_test, me_predict_lin) 
print(np.sqrt(lin_mse))


lin_perc = np.mean(100*np.divide(np.abs(me_test - me_predict_lin), me_test))
print(lin_perc)

#elasticnet = ElasticNet(alpha=0.01, l1_ratio=0.5)
#elasticnet.fit(poly_train, me_train)

#poly_predict = elasticnet.predict(poly_test)
#poly_mse = mean_squared_error(me_test, poly_predict)
#print(np.sqrt(poly_mse))

2.150695814584132e-14
8.230550588993383e-13
